# Training and testing classifires with MMMaction2
Hello! This script contains step by step learning path for neural networks I intend to teach. Good luck! :)

First let's initialize mmaction2 environment.

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.8.0+cu101 torchvision==0.9.0+cu101 torchtext==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.3.9 -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.0/index.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

##tsn_r50_video_1x1x8_100e_kinetics400_rgb



###Training

Firstly it's crucial to initiate the environment, in my case it's mmaction2, part of mmcv project.

Next, I have to mount my google drive, where I store my dataset which is made of about 2600 videos in 4 categories: breakdance, gymnastics tumbling, tricking and parkour.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#for training
!cp -arp /content/drive/MyDrive/computer_vision/my_set/ /content/mmaction2/data/



Time to initialize features used in teaching purposes.

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In order to shorten learning time, I will be using checkpoint of network prelearned on kinetisc400 dataset. The config file contains all of the learning and testing settings of chosen model.

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

!mkdir checkpoints

!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb/tsn_r50_video_1x1x8_100e_kinetics400_rgb_20200702-568cde33.pth \
      -O checkpoints/tsn_r50_video_1x1x8_100e_kinetics400_rgb_20200702-568cde33.pth


Now there is need for updating crucial data in config file.

In [ ]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'data/train/'
cfg.data_root_val = 'data/validate/'
cfg.ann_file_train = 'data/my_set_full_ann_train.txt'
cfg.ann_file_val = 'data/my_set_full_ann_val.txt'
cfg.ann_file_test = 'data/my_set_full_ann_val.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'data/my_set_full_ann_val.txt'
cfg.data.test.data_prefix = 'data/validate/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'data/my_set_full_ann_train.txt'
cfg.data.train.data_prefix = 'data/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'data/my_set_full_ann_val.txt'
cfg.data.val.data_prefix = 'data/validate/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 4
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsn_r50_video_1x1x8_100e_kinetics400_rgb_20200702-568cde33.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 30

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Finally, lets teach some networks, it will take time so make yourself comfortable. 

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

Download the result.

In [ ]:
from google.colab import files

!zip -r /content/my_checkpoint_full_first.zip /content/mmaction2/tutorial_exps

files.download('/content/my_checkpoint_full_first.zip')

##tsn_r50_video_1x1x8_100e_diving48_rgb



###Training

Firstly it's crucial to initiate the environment, in my case it's mmaction2, part of mmcv project.

Next, I have to mount my google drive, where I store my dataset which is made of about 2600 videos in 4 categories: breakdance, gymnastics tumbling, tricking and parkour.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


#for training

!mkdir /content/mmaction2/data
!mkdir checkpoints

!cp -arp /content/drive/MyDrive/computer_vision/my_set/train /content/mmaction2/data/
!cp -arp /content/drive/MyDrive/computer_vision/my_set/validate /content/mmaction2/data/
!cp -arp /content/drive/MyDrive/computer_vision/my_set/my_set_full_ann_train.txt /content/mmaction2/data/
!cp -arp /content/drive/MyDrive/computer_vision/my_set/my_set_full_ann_val.txt /content/mmaction2/data/

!cp -arp /content/drive/MyDrive/computer_vision/checkpoints/tsn_diving_epoch_15.pth /content/mmaction2/checkpoints/


Mounted at /content/drive


Time to initialize features used in teaching purposes.

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.8.0+cu101 True
0.17.0
10.1
GCC 7.3


In order to shorten learning time, I will be using checkpoint of network prelearned on kinetisc400 dataset. The config file contains all of the learning and testing settings of chosen model.

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_diving48_rgb.py')

# !mkdir checkpoints

!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_video_1x1x8_100e_diving48_rgb/tsn_r50_video_1x1x8_100e_diving48_rgb_20210426-6dde0185.pth \
      -O checkpoints/tsn_r50_video_1x1x8_100e_diving48_rgb_20210426-6dde0185.pth


--2021-08-20 07:28:06--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_video_1x1x8_100e_diving48_rgb/tsn_r50_video_1x1x8_100e_diving48_rgb_20210426-6dde0185.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.78
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97577830 (93M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_video_1x1x8_100e_diving48_rgb_20210426-6dde0185.pth’

checkpoints/tsn_r50 100%[===================>]  93.06M  12.3MB/s    in 7.9s    

2021-08-20 07:28:15 (11.8 MB/s) - ‘checkpoints/tsn_r50_video_1x1x8_100e_diving48_rgb_20210426-6dde0185.pth’ saved [97577830/97577830]



Now there is need for updating crucial data in config file.

In [ ]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'data/train/'
cfg.data_root_val = 'data/validate/'
cfg.ann_file_train = 'data/my_set_full_ann_train.txt'
cfg.ann_file_val = 'data/my_set_full_ann_val.txt'
cfg.ann_file_test = 'data/my_set_full_ann_val.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'data/my_set_full_ann_val.txt'
cfg.data.test.data_prefix = 'data/validate/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'data/my_set_full_ann_train.txt'
cfg.data.train.data_prefix = 'data/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'data/my_set_full_ann_val.txt'
cfg.data.val.data_prefix = 'data/validate/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 4
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsn_diving_epoch_15.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = 2
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 10

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Finally, lets teach some networks, it will take time so make yourself comfortable. 

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
#datasets = [build_dataset(cfg.data.train)]
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

Use load_from_torchvision loader


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

2021-08-20 07:28:19,205 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.bias', 'fc.weight'}
2021-08-20 07:28:30,436 - mmaction - INFO - load checkpoint from ./checkpoints/tsn_diving_epoch_15.pth
2021-08-20 07:28:30,438 - mmaction - INFO - Use load_from_local loader
2021-08-20 07:28:30,597 - mmaction - INFO - Start running, host: root@c7d995d864b6, work_dir: /content/mmaction2/tutorial_exps
2021-08-20 07:28:30,598 - mmaction - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -----------------

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 178/178, 2.4 task/s, elapsed: 75s, ETA:     0s

2021-08-20 08:56:51,029 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-08-20 08:56:51,031 - mmaction - INFO - 
top1_acc	0.8427
top5_acc	1.0000
2021-08-20 08:56:51,041 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-08-20 08:56:51,047 - mmaction - INFO - 
mean_acc	0.8382
2021-08-20 08:56:51,532 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_5.pth.
2021-08-20 08:56:51,533 - mmaction - INFO - Best top1_acc is 0.8427 at 5 epoch.
2021-08-20 08:56:51,534 - mmaction - INFO - Epoch(val) [5][89]	top1_acc: 0.8427, top5_acc: 1.0000, mean_class_accuracy: 0.8382
2021-08-20 08:56:59,347 - mmaction - INFO - Epoch [6][5/1265]	lr: 1.953e-05, eta: 1:27:04, time: 1.560, data_time: 1.359, memory: 1654, top1_acc: 0.7000, top5_acc: 1.0000, loss_cls: 0.6409, loss: 0.6409, grad_norm: 23.9822
2021-08-20 08:57:04,061 - mmaction - INFO - Epoch [6][10/1265]	lr: 1.953e-05, eta: 1:27:01, time: 0.944, data_time: 0.768, memory: 1654, top1_acc: 0.8000, top5_acc: 1.0000, los

Download the result.

In [ ]:
from google.colab import files

!zip -r /content/my_checkpoint_full_tsn_diving.zip /content/mmaction2/tutorial_exps

files.download('/content/my_checkpoint_full_tsn_diving.zip')

##tsm_r50_video_1x1x8_50e_diving48_rgb


###Training


Firstly it's crucial to initiate the environment, in my case it's mmaction2, part of mmcv project.

Next, I have to mount my google drive, where I store my dataset which is made of about 2600 videos in 4 categories: breakdance, gymnastics tumbling, tricking and parkour.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir data

#for training
!cp -arp /content/drive/MyDrive/computer_vision/my_set/train /content/mmaction2/data/train/
!cp -arp /content/drive/MyDrive/computer_vision/my_set/validate /content/mmaction2/data/validate/
!cp -arp /content/drive/MyDrive/computer_vision/my_set/my_set_full_ann_train.txt  /content/mmaction2/data/
!cp -arp /content/drive/MyDrive/computer_vision/my_set/my_set_full_ann_val.txt  /content/mmaction2/data/

Mounted at /content/drive


Time to initialize features used in teaching purposes.

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.8.0+cu101 True
0.17.0
10.1
GCC 7.3


In order to shorten learning time, I will be using checkpoint of network prelearned on kinetisc400 dataset. The config file contains all of the learning and testing settings of chosen model.

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsm/tsm_r50_video_1x1x8_50e_diving48_rgb.py')

!mkdir checkpoints

!cp -arp /content/drive/MyDrive/computer_vision/checkpoints/tsm_diving_epoch_10.pth /content/mmaction2/checkpoints/

!wget -c https://download.openmmlab.com/mmaction/recognition/tsm/tsm_r50_video_1x1x8_50e_diving48_rgb/tsm_r50_video_1x1x8_50e_diving48_rgb_20210426-aba5aa3d.pth \
      -O checkpoints/tsm_r50_video_1x1x8_50e_diving48_rgb_20210426-aba5aa3d.pth

--2021-08-28 10:10:29--  https://download.openmmlab.com/mmaction/recognition/tsm/tsm_r50_video_1x1x8_50e_diving48_rgb/tsm_r50_video_1x1x8_50e_diving48_rgb_20210426-aba5aa3d.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.78
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94692905 (90M) [application/octet-stream]
Saving to: ‘checkpoints/tsm_r50_video_1x1x8_50e_diving48_rgb_20210426-aba5aa3d.pth’

checkpoints/tsm_r50 100%[===================>]  90.31M  7.96MB/s    in 9.7s    

2021-08-28 10:10:40 (9.34 MB/s) - ‘checkpoints/tsm_r50_video_1x1x8_50e_diving48_rgb_20210426-aba5aa3d.pth’ saved [94692905/94692905]



Now there is need for updating crucial data in config file.

In [ ]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'data/train/'
cfg.data_root_val = 'data/validate/'
cfg.ann_file_train = 'data/my_set_full_ann_train.txt'
cfg.ann_file_val = 'data/my_set_full_ann_val.txt'
cfg.ann_file_test = 'data/my_set_full_ann_val.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'data/my_set_full_ann_val.txt'
cfg.data.test.data_prefix = 'data/validate/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'data/my_set_full_ann_train.txt'
cfg.data.train.data_prefix = 'data/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'data/my_set_full_ann_val.txt'
cfg.data.val.data_prefix = 'data/validate/'


# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 4
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsm_diving_epoch_10.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = 2
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 10

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 2
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNetTSM',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False,
        shift_div=8),
    cls_head=dict(
        type='TSMHead',
        num_classes=4,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.5,
        init_std=0.001,
        is_shift=True),
    train_cfg=None,
    test_cfg=dict(average_clips='prob'))
optimizer = dict(
    type='SGD',
    constructor='TSMOptimizerConstructor',
    paramwise_cfg=dict(fc_lr5=True),
    lr=7.8125e-05,
    momentum=0.9,
    weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=20, norm_type=2))
lr_config = dict(policy='step', step=[20, 40])
total_epochs = 10
checkpoint_config = dict(interval=2)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = './checkpoint

Finally, lets teach some networks, it will take time so make yourself comfortable. 

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
#datasets = [build_dataset(cfg.data.train)]
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

Use load_from_torchvision loader


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

2021-08-28 10:10:44,544 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.bias', 'fc.weight'}
2021-08-28 10:10:55,833 - mmaction - INFO - load checkpoint from ./checkpoints/tsm_diving_epoch_10.pth
2021-08-28 10:10:55,835 - mmaction - INFO - Use load_from_local loader
2021-08-28 10:10:56,000 - mmaction - INFO - Start running, host: root@adba58358be3, work_dir: /content/mmaction2/tutorial_exps
2021-08-28 10:10:56,001 - mmaction - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -----------------

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 178/178, 2.4 task/s, elapsed: 73s, ETA:     0s

...

2021-08-28 12:38:58,398 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-08-28 12:38:58,402 - mmaction - INFO - 
top1_acc	0.8820
top5_acc	1.0000
2021-08-28 12:38:58,407 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-08-28 12:38:58,415 - mmaction - INFO - 
mean_acc	0.8800
2021-08-28 12:38:58,419 - mmaction - INFO - Epoch(val) [8][89]	top1_acc: 0.8820, top5_acc: 1.0000, mean_class_accuracy: 0.8800
2021-08-28 12:39:05,987 - mmaction - INFO - Epoch [9][5/1265]	lr: 7.813e-05, eta: 0:34:32, time: 1.512, data_time: 1.261, memory: 2000, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.8739, loss: 0.8739, grad_norm: 37.8454
2021-08-28 12:39:09,559 - mmaction - INFO - Epoch [9][10/1265]	lr: 7.813e-05, eta: 0:34:27, time: 0.713, data_time: 0.469, memory: 2000, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.3655, loss: 0.3655, grad_norm: 20.2050
2021-08-28 12:39:14,226 - mmaction - INFO - Epoch [9][15/1265]	lr: 7.813e-05, eta: 0:34:24, time: 0.934, data_time: 0.692, memory: 2000

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 178/178, 2.4 task/s, elapsed: 73s, ETA:     0s

2021-08-28 12:57:45,131 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-08-28 12:57:45,133 - mmaction - INFO - 
top1_acc	0.8933
top5_acc	1.0000
2021-08-28 12:57:45,143 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-08-28 12:57:45,145 - mmaction - INFO - 
mean_acc	0.8894
2021-08-28 12:57:45,149 - mmaction - INFO - Epoch(val) [9][89]	top1_acc: 0.8933, top5_acc: 1.0000, mean_class_accuracy: 0.8894
2021-08-28 12:57:51,335 - mmaction - INFO - Epoch [10][5/1265]	lr: 7.813e-05, eta: 0:17:15, time: 1.236, data_time: 0.998, memory: 2000, top1_acc: 1.0000, top5_acc: 1.0000, loss_cls: 0.0820, loss: 0.0820, grad_norm: 11.9381
2021-08-28 12:57:54,439 - mmaction - INFO - Epoch [10][10/1265]	lr: 7.813e-05, eta: 0:17:11, time: 0.621, data_time: 0.384, memory: 2000, top1_acc: 1.0000, top5_acc: 1.0000, loss_cls: 0.0989, loss: 0.0989, grad_norm: 12.1583
2021-08-28 12:57:58,087 - mmaction - INFO - Epoch [10][15/1265]	lr: 7.813e-05, eta: 0:17:07, time: 0.730, data_time: 0.500, memory: 2

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 178/178, 2.5 task/s, elapsed: 72s, ETA:     0s

2021-08-28 13:16:17,805 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-08-28 13:16:17,808 - mmaction - INFO - 
top1_acc	0.8933
top5_acc	1.0000
2021-08-28 13:16:17,816 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-08-28 13:16:17,818 - mmaction - INFO - 
mean_acc	0.8929
2021-08-28 13:16:17,821 - mmaction - INFO - Epoch(val) [10][89]	top1_acc: 0.8933, top5_acc: 1.0000, mean_class_accuracy: 0.8929


Download the result.

In [ ]:
from google.colab import files

!zip -r /content/my_checkpoint_full_first.zip /content/mmaction2/tutorial_exps

files.download('/content/my_checkpoint_full_first.zip')

  adding: content/mmaction2/tutorial_exps/ (stored 0%)
  adding: content/mmaction2/tutorial_exps/epoch_8.pth (deflated 7%)
  adding: content/mmaction2/tutorial_exps/.ipynb_checkpoints/ (stored 0%)
  adding: content/mmaction2/tutorial_exps/epoch_10.pth (deflated 7%)
  adding: content/mmaction2/tutorial_exps/None.log.json (deflated 87%)
  adding: content/mmaction2/tutorial_exps/latest.pth (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##tsm_r50_video_1x1x8_50e_kinetics400_rgb


###Training


Firstly it's crucial to initiate the environment, in my case it's mmaction2, part of mmcv project.

Next, I have to mount my google drive, where I store my dataset which is made of about 2600 videos in 4 categories: breakdance, gymnastics tumbling, tricking and parkour.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd mmaction2/
!mkdir data

#for training
!cp -arp /content/drive/MyDrive/computer_vision/my_set/train /content/mmaction2/data/train/
!cp -arp /content/drive/MyDrive/computer_vision/my_set/validate /content/mmaction2/data/validate/
!cp -arp /content/drive/MyDrive/computer_vision/my_set/my_set_full_ann_train.txt  /content/mmaction2/data/
!cp -arp /content/drive/MyDrive/computer_vision/my_set/my_set_full_ann_val.txt  /content/mmaction2/data/

Time to initialize features used in teaching purposes.

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In order to shorten learning time, I will be using checkpoint of network prelearned on kinetisc400 dataset. The config file contains all of the learning and testing settings of chosen model.

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsm/tsm_r50_video_1x1x8_50e_kinetics400_rgb.py')

!mkdir checkpoints

!wget -c https://download.openmmlab.com/mmaction/recognition/tsm/tsm_r50_video_1x1x8_100e_kinetics400_rgb/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth \
      -O checkpoints/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth


mkdir: cannot create directory ‘checkpoints’: File exists
--2021-08-26 11:09:59--  https://download.openmmlab.com/mmaction/recognition/tsm/tsm_r50_video_1x1x8_100e_kinetics400_rgb/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.254.186.225
Connecting to download.openmmlab.com (download.openmmlab.com)|47.254.186.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97579687 (93M) [application/octet-stream]
Saving to: ‘checkpoints/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth’

checkpoints/tsm_r50 100%[===================>]  93.06M  11.1MB/s    in 9.6s    

2021-08-26 11:10:10 (9.69 MB/s) - ‘checkpoints/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth’ saved [97579687/97579687]



Now there is need for updating crucial data in config file.

In [ ]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'data/train/'
cfg.data_root_val = 'data/validate/'
cfg.ann_file_train = 'data/my_set_full_ann_train.txt'
cfg.ann_file_val = 'data/my_set_full_ann_val.txt'
cfg.ann_file_test = 'data/my_set_full_ann_val.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'data/my_set_full_ann_val.txt'
cfg.data.test.data_prefix = 'data/validate/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'data/my_set_full_ann_train.txt'
cfg.data.train.data_prefix = 'data/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'data/my_set_full_ann_val.txt'
cfg.data.val.data_prefix = 'data/validate/'


# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 4
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = 2
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 20

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNetTSM',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False,
        shift_div=8),
    cls_head=dict(
        type='TSMHead',
        num_classes=4,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.5,
        init_std=0.001,
        is_shift=True),
    train_cfg=None,
    test_cfg=dict(average_clips='prob'))
optimizer = dict(
    type='SGD',
    constructor='TSMOptimizerConstructor',
    paramwise_cfg=dict(fc_lr5=True),
    lr=0.00015625,
    momentum=0.9,
    weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=20, norm_type=2))
lr_config = dict(policy='step', step=[20, 40])
total_epochs = 20
checkpoint_config = dict(interval=10)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = './checkpoin

Finally, lets teach some networks, it will take time so make yourself comfortable. 

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
#datasets = [build_dataset(cfg.data.train)]
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

Use load_from_torchvision loader


2021-08-26 11:10:33,740 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.bias', 'fc.weight'}
2021-08-26 11:10:33,794 - mmaction - INFO - load checkpoint from ./checkpoints/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth
2021-08-26 11:10:33,799 - mmaction - INFO - Use load_from_local loader
2021-08-26 11:10:33,898 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([4, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([4]).
2021-08-26 11:10:33,902 - mmaction - INFO - Start running, host: root@3255f9c8ce46, work_dir: /content/mmaction2/tutorial_exps
2021-08-26 11:10:33,908 - mmaction - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) St

KeyboardInterrupt: ignored

Download the result.

In [ ]:
from google.colab import files

!zip -r /content/my_checkpoint_full_first.zip /content/mmaction2/tutorial_exps

files.download('/content/my_checkpoint_full_first.zip')

#Plotting and analisys

Plot and analyse data 

In [ ]:
from google.colab import drive
from mmcv import Config

drive.mount('/content/drive')

!mkdir checkpoints

!cp -arp /content/drive/MyDrive/computer_vision/checkpoints/tsm_diving_epoch_20.pth /content/mmaction2/checkpoints/
!cp -arp /content/drive/MyDrive/computer_vision/checkpoints/tsm_kinetics400_epoch_20.pth /content/mmaction2/checkpoints/
!cp -arp /content/drive/MyDrive/computer_vision/checkpoints/tsn_diving_epoch_20.pth /content/mmaction2/checkpoints/
!cp -arp /content/drive/MyDrive/computer_vision/checkpoints/tsn_kinetics400_epoch_20.pth /content/mmaction2/checkpoints/
!cp -arp /content/drive/MyDrive/computer_vision/configs/graduation2/ /content/mmaction2/configs/

#VALIDATION
#!cp -apr /content/drive/MyDrive/computer_vision/my_set/validate /content/mmaction2/data/
#!cp -apr /content/drive/MyDrive/computer_vision/my_set/my_set_full_ann_val.txt /content/mmaction2/data/

#TEST
!cp -apr /content/drive/MyDrive/computer_vision/my_set/test/ /content/mmaction2/data/
!cp -apr /content/drive/MyDrive/computer_vision/my_set/my_set_full_ann_test.txt /content/mmaction2/data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘checkpoints’: File exists


In [ ]:
!python tools/test.py configs/graduation2/tsm/tsm_r50_video_1x1x8_50e_kinetics400_rgb_modified.py checkpoints/tsm_kinetics400_epoch_20.pth --eval top_k_accuracy 
!python tools/test.py configs/graduation2/tsm/tsm_r50_video_1x1x8_50e_diving48_rgb_modified.py checkpoints/tsm_diving_epoch_20.pth --eval top_k_accuracy 
!python tools/test.py configs/graduation2/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb_modified.py checkpoints/tsn_kinetics400_epoch_20.pth --eval top_k_accuracy 
!python tools/test.py configs/graduation2/tsn/tsn_r50_video_1x1x8_100e_diving48_rgb_modified.py checkpoints/tsn_diving_epoch_20.pth --eval top_k_accuracy 

In [ ]:
!python tools/test.py configs/graduation2/tsm/tsm_r50_video_1x1x8_50e_kinetics400_rgb_modified.py checkpoints/tsm_kinetics400_epoch_20.pth --eval mean_class_accuracy 
!python tools/test.py configs/graduation2/tsm/tsm_r50_video_1x1x8_50e_diving48_rgb_modified.py checkpoints/tsm_diving_epoch_20.pth --eval mean_class_accuracy 
!python tools/test.py configs/graduation2/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb_modified.py checkpoints/tsn_kinetics400_epoch_20.pth --eval mean_class_accuracy 
!python tools/test.py configs/graduation2/tsn/tsn_r50_video_1x1x8_100e_diving48_rgb_modified.py checkpoints/tsn_diving_epoch_20.pth --eval mean_class_accuracy 

In [ ]:
from google.colab import drive
from mmcv import Config

drive.mount('/content/drive', force_remount=True)
!python tools/analysis/analyze_logs.py plot_curve /content/drive/MyDrive/computer_vision/welded_logs_poprawione.json --keys loss_cls --legend loss_cls --out loss_cls.png
!python tools/analysis/analyze_logs.py plot_curve /content/drive/MyDrive/computer_vision/welded_logs_poprawione.json --keys top1_acc --legend top1_acc --out top1_acc.png

Mounted at /content/drive
plot curve of /content/drive/MyDrive/computer_vision/welded_logs_poprawione.json, metric is loss_cls
save curve to: loss_cls.png
plot curve of /content/drive/MyDrive/computer_vision/welded_logs_poprawione.json, metric is top1_acc
save curve to: top1_acc.png
